# LIBARY

In [1]:
!pip install geopandas fiona pyogrio
!pip install geopandas

In [3]:
import geopandas as gpd
import fiona
import pyogrio
import os

# READ BUILDING DATA

In [21]:
cwd = os.getcwd()

# get parent directory
parent_dir = os.path.dirname(cwd)


# IDAHo


# building path
path = os.path.join(parent_dir, 'Data', 'building_data_gdb', 'CA_Structures.gdb')


# read the data
gdf = gpd.read_file(path)

In [22]:
gdf.head()

,BUILD_ID,OCC_CLS,PRIM_OCC,SEC_OCC,PROP_ADDR,PROP_CITY,PROP_ST,PROP_ZIP,OUTBLDG,HEIGHT,...,LONGITUDE,LATITUDE,IMAGE_NAME,IMAGE_DATE,VAL_METHOD,REMARKS,UUID,Shape_Length,Shape_Area,geometry
0,2711218,Agriculture,Agriculture,None,None,None,California,None,None,NaN,...,-124.180751,40.132823,103001008E778300,2019-02-21 00:00:00+00:00,Automated,None,{961790f0-0a8a-4b73-8dea-ce9bb27bd44f},0.000616,2.331285e-08,"MULTIPOLYGON (((-124.18085 40.13276, -124.1808..."
1,2712763,Education,Pre-K - 12 Schools,None,None,None,California,None,None,NaN,...,-124.213215,40.209654,1040010012522900,2015-10-13 00:00:00+00:00,Automated,None,{bdaa93ed-c485-4c1a-919b-32c7afce9770},0.000308,5.834568e-09,"MULTIPOLYGON (((-124.21317 40.20962, -124.2132..."
2,2712975,Residential,Single Family Dwelling,None,None,None,California,None,None,NaN,...,-124.189050,40.222208,103001008E778300,2019-02-21 00:00:00+00:00,Automated,None,{df86d319-d7a6-476f-8a02-615701f888af},0.000308,5.835813e-09,"MULTIPOLYGON (((-124.18909 40.22224, -124.189 ..."
3,2713048,Residential,Single Family Dwelling,None,None,None,California,None,None,NaN,...,-124.251666,40.225007,1040010012522900,2015-10-13 00:00:00+00:00,Automated,None,{7bb67d5f-ac6c-4ade-af24-f9ea573536bd},0.000407,9.726265e-09,"MULTIPOLYGON (((-124.25159 40.22504, -124.2516..."
4,2713052,Education,Pre-K - 12 Schools,None,None,None,California,None,None,NaN,...,-124.247156,40.225118,1040010012522900,2015-10-13 00:00:00+00:00,Automated,None,{fa63a760-7b7b-4b81-abce-4fc67f5d40f2},0.000471,1.361703e-08,"MULTIPOLYGON (((-124.2472 40.22505, -124.2472 ..."


In [23]:
building_data = gdf[['BUILD_ID', 'OCC_CLS', 'PRIM_OCC', 'SQFEET', 'FIPS', 'CENSUSCODE', 'LONGITUDE', 'LATITUDE', 'Shape_Length', 'Shape_Area',
                    'geometry']]

In [24]:
# change building data to geodataframe
building_data = gpd.GeoDataFrame(building_data, geometry='geometry')
len(building_data)

10931401

In [26]:
building_data.head()

,BUILD_ID,OCC_CLS,PRIM_OCC,SQFEET,FIPS,CENSUSCODE,LONGITUDE,LATITUDE,Shape_Length,Shape_Area,geometry
0,2711218,Agriculture,Agriculture,2588.805176,06023,06023011200,-124.180751,40.132823,0.000616,2.331285e-08,"MULTIPOLYGON (((-124.18085 40.13276, -124.1808..."
1,2712763,Education,Pre-K - 12 Schools,484.318878,06023,06023011200,-124.213215,40.209654,0.000308,5.834568e-09,"MULTIPOLYGON (((-124.21317 40.20962, -124.2132..."
2,2712975,Residential,Single Family Dwelling,531.596191,06023,06023011200,-124.189050,40.222208,0.000308,5.835813e-09,"MULTIPOLYGON (((-124.18909 40.22224, -124.189 ..."
3,2713048,Residential,Single Family Dwelling,857.398621,06023,06023011200,-124.251666,40.225007,0.000407,9.726265e-09,"MULTIPOLYGON (((-124.25159 40.22504, -124.2516..."
4,2713052,Education,Pre-K - 12 Schools,1376.912598,06023,06023011200,-124.247156,40.225118,0.000471,1.361703e-08,"MULTIPOLYGON (((-124.2472 40.22505, -124.2472 ..."


In [46]:
building_data['OCC_CLS'].unique()

# mapping the occupancy class
mapping = {
    'Agriculture':'OTHER', 'Education':'OTHER', 'Residential':'RESIDENTIAL', 'Unclassified':'OTHER',
    'Commercial':'OTHER', 'Government':'OTHER', 'Industrial':'OTHER', 'Utility and Misc':'OTHER',
    'Assembly':'OTHER'
}
building_data['OCC_CLS'] = building_data['OCC_CLS'].map(mapping)

In [47]:
# mapping the primary occupancy
building_data['PRIM_OCC'].unique()

array(['Agriculture', 'Pre-K - 12 Schools', 'Single Family Dwelling',
       'Unclassified', 'Manufactured Home', 'Retail Trade',
       'Emergency Response', 'Entertainment and Recreation',
       'Temporary Lodging', 'Multi - Family Dwelling',
       'Other Educational Buildings', 'Professional/Technical Services',
       'Parking', 'Light', 'General Services',
       'Personal and Repair Services', 'Colleges/Universities', 'Ground',
       'Religious', 'Nursing Home', 'Heavy', 'Wholesale Trade',
       'Medical Office/Clinic', 'Metals/Minerals Processing', 'Aviation',
       'Institutional Dormitory', 'Non-Civilian Structures',
       'Community Center', 'Theaters', 'Food/Drugs/Chemicals', 'Hospital',
       'Energy Control Monitoring', 'Indoor Arena', 'High Technology',
       'Veterinary/Pet ', 'Banks', 'Marine', 'Rail', 'Construction',
       'Stadium'], dtype=object)

In [53]:
# make mapping dictionary through looping
mapping = {i:'OTHER' for i in building_data['PRIM_OCC'].unique() if i not in ['Single Family Dwelling', 'Multi - Family Dwelling']}
residential = {'Single Family Dwelling':'SINGLE FAMILY', 'Multi - Family Dwelling':'MULTI FAMILY'}
mapping.update(residential)
mapping

{'Agriculture': 'OTHER',
 'Pre-K - 12 Schools': 'OTHER',
 'Unclassified': 'OTHER',
 'Manufactured Home': 'OTHER',
 'Retail Trade': 'OTHER',
 'Emergency Response': 'OTHER',
 'Entertainment and Recreation': 'OTHER',
 'Temporary Lodging': 'OTHER',
 'Other Educational Buildings': 'OTHER',
 'Professional/Technical Services': 'OTHER',
 'Parking': 'OTHER',
 'Light': 'OTHER',
 'General Services': 'OTHER',
 'Personal and Repair Services': 'OTHER',
 'Colleges/Universities': 'OTHER',
 'Ground': 'OTHER',
 'Religious': 'OTHER',
 'Nursing Home': 'OTHER',
 'Heavy': 'OTHER',
 'Wholesale Trade': 'OTHER',
 'Medical Office/Clinic': 'OTHER',
 'Metals/Minerals Processing': 'OTHER',
 'Aviation': 'OTHER',
 'Institutional Dormitory': 'OTHER',
 'Non-Civilian Structures': 'OTHER',
 'Community Center': 'OTHER',
 'Theaters': 'OTHER',
 'Food/Drugs/Chemicals': 'OTHER',
 'Hospital': 'OTHER',
 'Energy Control Monitoring': 'OTHER',
 'Indoor Arena': 'OTHER',
 'High Technology': 'OTHER',
 'Veterinary/Pet ': 'OTHER',
 'B

In [54]:
building_data['PRIM_OCC'] = building_data['PRIM_OCC'].map(mapping)
building_data['PRIM_OCC'].unique()

array(['OTHER', 'SINGLE FAMILY', 'MULTI FAMILY'], dtype=object)

In [55]:
building_data.groupby(['CENSUSCODE', 'OCC_CLS', 'PRIM_OCC'])['BUILD_ID'].count()

CENSUSCODE   OCC_CLS      PRIM_OCC     
06001400100  OTHER        OTHER             104
             RESIDENTIAL  MULTI FAMILY       22
                          OTHER              10
                          SINGLE FAMILY    1162
06001400200  OTHER        OTHER              46
                                           ... 
06115041101  RESIDENTIAL  SINGLE FAMILY     903
06115041102  OTHER        OTHER             189
             RESIDENTIAL  MULTI FAMILY       67
                          OTHER             354
                          SINGLE FAMILY     474
Name: BUILD_ID, Length: 34201, dtype: int64

# READ THE TRACT DATA

In [25]:

# function to read census tract data based on state id
def read_census_tract_data(state_id):
    """
    Function to read census tract data based on state id.
    It will read the census tract data from census_tracts folder.
    """
    # Read states census tract shapefile
    tract_path = os.path.join(parent_dir, 'Data', 'tract', f'tl_2024_{state_id}_tract', f'tl_2024_{state_id}_tract.shp')
    gdf_tract = gpd.read_file(tract_path)
    
    # Display the first few rows of the GeoDataFrame
    return gdf_tract

In [35]:
gdf_tract = read_census_tract_data('06')

In [36]:
gdf_tract_geometry = gdf_tract[['GEOID', 'geometry']]

In [37]:
# change gdf_tract_geometry to geodataframe
gdf_tract_geometry = gpd.GeoDataFrame(gdf_tract_geometry, geometry='geometry')

# INTERSECT BUILDING AND CENSUS TRACT

In [38]:
building_data.to_crs(epsg=4326, inplace=True)
gdf_tract_geometry.to_crs(epsg=4326, inplace=True)

In [39]:
# spatial join between building data and census tract data
building_data_tract = gpd.sjoin(building_data, gdf_tract_geometry, how='left', predicate='intersects')

In [40]:
# make sure same crs, change the crs of building data to census tract data
building_data_tract.isnull().sum()

BUILD_ID         0
OCC_CLS          0
PRIM_OCC         0
SQFEET           0
FIPS             0
CENSUSCODE       0
LONGITUDE        0
LATITUDE         0
Shape_Length     0
Shape_Area       0
geometry         0
index_right     11
GEOID           11
dtype: int64

In [41]:
building_data_tract

,BUILD_ID,OCC_CLS,PRIM_OCC,SQFEET,FIPS,CENSUSCODE,LONGITUDE,LATITUDE,Shape_Length,Shape_Area,geometry,index_right,GEOID
0,2711218,Agriculture,Agriculture,2588.805176,06023,06023011200,-124.180751,40.132823,0.000616,2.331285e-08,"MULTIPOLYGON (((-124.18085 40.13276, -124.1808...",5349.0,06023011200
1,2712763,Education,Pre-K - 12 Schools,484.318878,06023,06023011200,-124.213215,40.209654,0.000308,5.834568e-09,"MULTIPOLYGON (((-124.21317 40.20962, -124.2132...",5349.0,06023011200
2,2712975,Residential,Single Family Dwelling,531.596191,06023,06023011200,-124.189050,40.222208,0.000308,5.835813e-09,"MULTIPOLYGON (((-124.18909 40.22224, -124.189 ...",5349.0,06023011200
3,2713048,Residential,Single Family Dwelling,857.398621,06023,06023011200,-124.251666,40.225007,0.000407,9.726265e-09,"MULTIPOLYGON (((-124.25159 40.22504, -124.2516...",5349.0,06023011200
4,2713052,Education,Pre-K - 12 Schools,1376.912598,06023,06023011200,-124.247156,40.225118,0.000471,1.361703e-08,"MULTIPOLYGON (((-124.2472 40.22505, -124.2472 ...",5349.0,06023011200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10931396,10008883,Residential,Nursing Home,60672.976562,06037,06037106408,-118.439755,34.311521,0.007765,5.278148e-07,"MULTIPOLYGON (((-118.43986 34.31201, -118.4395...",4216.0,06037106408
10931397,10008884,Education,Pre-K - 12 Schools,88355.734375,06037,06037106405,-118.449916,34.311938,0.007817,7.969720e-07,"MULTIPOLYGON (((-118.44963 34.31189, -118.4495...",7041.0,06037106405
10931398,10008885,Industrial,Heavy,291899.625000,06037,06037106510,-118.474949,34.313049,0.007847,2.657830e-06,"MULTIPOLYGON (((-118.47466 34.31217, -118.4748...",1949.0,06037106510
10931399,10008886,Education,Pre-K - 12 Schools,136062.750000,06037,06037106405,-118.449563,34.318507,0.008448,1.234481e-06,"MULTIPOLYGON (((-118.44935 34.31838, -118.4494...",7041.0,06037106405


# AGGREGATE DATA

In [ ]:
# get count building per census tract
building_count = building_data_tract.groupby('GEOID').size().reset_index(name='count')

# save the data as csv
building_count.to_csv(os.path.join(parent_dir, 'Data', 'building_count.csv'), index=False)